#   Task 1 - Disease Named Entity Recognition (D-NER)
## Fitting the Data

Note: All the data too large for github, will be available for download through a link in the repository discription.

## Import

In [1]:
import pandas as pd
import os
from nltk.corpus import stopwords
from nltk.util import ngrams
import string
import scipy
import joblib
import numpy as np
import torch
from typing import List
from torch.utils.data import TensorDataset
from transformers import AutoTokenizer


/home/florian/.pyenv/versions/3.7.15/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-28 15:54:49.885856: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 15:54:51.055722: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-28 15:54:51.056011: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_p

In [2]:
#Now for the import we create the soon to be objects:
DC1 = []
DC2 = []
SD1 = []
SD2 = []
SD3 = []
Mod = []

def file_open(path, variable):
    with open(path, 'r') as fp:
        for line in fp:
            # remove linebreak from a current name, linebreak is the last character of each line
            x = line[:-1] # add current item to the list
            variable.append(x)
            return variable

DC1 = file_open('./data/DC1.txt', DC1)
DC1 = file_open('./data/DC2.txt', DC2)
DC1 = file_open('./data/Mod.txt', Mod)
DC1 = file_open('./data/SD1.txt', SD1)
DC1 = file_open('./data/SD2.txt', SD2)
DC1 = file_open('./data/SD3.txt', SD3)

## Transformations and Matching

For feeding the MIMIC III data to our labeling functions we opted for a n-gram approach - i. e. every note entry is split into x n-grams corresponding to y number of rows. As a starting point we opted for a selection of unigrams. Yet the keyword lists prepared would hold the potential to make for a higher accuracy with bigger performance needs. For computational efficiency, those will be imported and exported.

In [2]:
chunksize = 10000
#notes_text = pd.read_csv('./data/notes_cleaned.csv', low_memory=False, chunksize = chunksize, index_col = False)

In [3]:
# processing ngrams in chunks for performance reasons


def get_ngrams(file_path, n, df):
    try:
        os.remove(file_path)
    except OSError:
        pass

    for subset in df:
        subset = subset.dropna()
        subset['ngrams'] = subset['TEXT'].str.split().apply(lambda x: list(map(' '.join, ngrams(x, n=n))))
        subset = (subset.assign(count=subset['ngrams'].str.len())
    .explode('ngrams')
    .query('count > 0'))
        subset['index_notes'] = subset.index
        subset = subset.drop(['count', 'TEXT'], axis=1)
        if not os.path.isfile(file_path):
            subset.to_csv(file_path, index=False)
        else:
            subset.to_csv(file_path, index=False, mode='a', header=False)
    return None

In [15]:
#get_ngrams('./data/notes_1grams.csv', 1, notes_text)

#   Labeling functions

Next we label our testdata ngrams with the previously extracted keywords, returning a dataframe for further use.

In [4]:
# function for extracting a subset of the relevant keywords from a keyword list, relevant to the given ngrams to map them to.

def ngrams_keywords(n, input_ls):
    output_ls = []
    for i in input_ls:
        ws = i.count(' ')
        if ws == n-1:
            output_ls.append(i)
    return output_ls

In [111]:
def labeling_function(file_path, df, keyword_list):
    """labeling function takes as input a dataframe and a list of keywords and encodes every ngram with 0/1 columns for all the possible labels (1 if exact match, else 0)"""
    try:
        os.remove(file_path)
    except OSError:
        pass

    for subset in df:
        #subset = subset.dropna()
        for keyword in keyword_list:
            subset[keyword] = subset['ngrams'].map(lambda x: 1 if x == keyword  else 0)
    if not os.path.isfile(file_path):
        subset.to_csv(file_path, index=False)
    else:
        subset.to_csv(file_path, index=False, mode='a', header=False)
    return None

In [5]:
#notes_ngrams = pd.read_csv('./data/notes_1grams.csv')

Reorganizing and renaming of the columns

Given the continuing struggles with the size, dimensionality and computational demands, we have settled on opting for a small prove-of-concept subset of the first set of all keyword lists and of each of them only the unigrams. But the existing functions are flexible enough to expand in scope.

In [94]:
#labeling_function('/Storage/Data/Knodle/notes_ngrams_labeled.csv', notes_ngrams,keyword_list)

/home/florian/.pyenv/versions/3.7.15/lib/python3.7/site-packages/ipykernel_launcher.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if sys.path[0] == "":


In [113]:
notes_ngrams_labeled=pd.read_csv('./data/notes_ngrams_labeled.csv')

#### Should this project be expanded upon and more than unigrams used, the following function would expand upon the output of the labeling function to provide for scaleability options, in order to keep the training data format consistent and normalized over larger ngram selection.

In [ ]:
# def explode_ngrams(file_path, df):
#     """takes as input a dataframe of ngrams and creates for every word within a ngram a separate row, then the columns of the newly generated dataframe are also rearranged and renamed for clearer formatting"""
#     try:
#         os.remove(file_path)
#     except OSError:
#         pass

#     for subset in df:
#         subset['words'] = subset['ngrams'].str.split(' ')
#         subset = subset.explode('words')

#         cols = subset.columns.tolist()
#         cols = cols[-1:] + cols[:-1]

#         subset = subset[cols]

#         subset = subset.rename(columns={'ngrams':'keywords'})

#         if not os.path.isfile(file_path):
#             subset.to_csv(file_path, index=False)
#         else:
#             subset.to_csv(file_path, index=False, mode='a', header=False)

# Input Matrices

In the following the relevant input mnatrices are created and exported in joblib format for later use.

## T and Z Matrix

In [112]:
DC1_uni = list(set(ngrams_keywords(1, DC1)))
SD1_uni = list(set(ngrams_keywords(1, SD1)))

DC1_uni = [x for x in DC1_uni if x]
SD1_uni = [x for x in SD1_uni if x]

keyword_list = list(set(DC1_uni + SD1_uni))
len(keyword_list)

574

T Matrix Creation:

dimensions keywords x labels

In [80]:
df_T = pd.DataFrame(keyword_list)
df_T = df_T.rename(columns={df_T.columns[0]:'Keywords'})
df_T['DC'] = df_T['Keywords'].map(lambda x: 1 if x in DC1_uni else 0)
df_T['SD'] = df_T['Keywords'].map(lambda x: 1 if x in SD1_uni else 0)

t_matrix = df_T.set_index('Keywords')
t_matrix = scipy.sparse.csr_matrix(t_matrix)

In [84]:
t_matrix = df_T.set_index('Keywords')
t_matrix = scipy.sparse.csr_matrix(t_matrix)
t_matrix = t_matrix.tocoo()

In [60]:
t_matrix = torch.sparse.LongTensor(torch.LongTensor([t_matrix.row.tolist(), t_matrix.col.tolist()]), torch.LongTensor(t_matrix.data.astype(np.int32)))

In [61]:
joblib.dump(t_matrix, './data/t_matrix.lib')

['/Storage/Data/Knodle/t_matrix.lib']

In [1]:
notes_ngrams_labeled = pd.read_csv('./data/notes_ngrams_labeled.csv', low_memory=False, chunksize = 500000, index_col = False)

NameError: name 'pd' is not defined

In [155]:
ls_matrix = []
for subset in notes_ngrams_labeled:
    try:
        subset = subset.drop('index_notes', axis=1)
        subset = subset.set_index('ngrams')
        temp_matrix = scipy.sparse.csr_matrix(subset)
        ls_matrix.append(temp_matrix)
    except:
        pass 

In [156]:
joblib.dump(ls_matrix, './data/ls_matrix.lib')

['/Storage/Data/Knodle/ls_matrix.lib']

In [145]:
ls_matrix = joblib.load('./data/ls_matrix.lib')

Z Matrix

dimensions: ngrams x keywords and an additional index column for the respective clinical note from which a given ngram stems

In [166]:
z_matrix = scipy.sparse.hstack(ls_matrix)
z_matrix.get_shape()

(412685, 574)

In [167]:
z_matrix = z_matrix.tocoo()

values = z_matrix.data
indices = np.vstack((z_matrix.row, z_matrix.col))

i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = z_matrix.shape

z_matrix=torch.sparse.FloatTensor(i, v, torch.Size(shape)).to_dense()
z_matrix.size()

torch.Size([412685, 574])

In [168]:
joblib.dump(z_matrix, './data/z_matrix.lib')

['/Storage/Data/Knodle/z_matrix.lib']

In [33]:
#z_matrix = joblib.load('/Storage/Data/Knodle/z_matrix.lib')

#  Final Preparations for the test and dev data

In [13]:
test_fin = pd.read_csv('./data/test_fin.csv')
dev_fin = pd.read_csv('./data/dev_fin.csv')
dev_fin.dropna(inplace=True)

In [14]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Tokenizing, as well as translating the existing classes into integers, with the help of transformers and the scipy package

In [15]:
def convert_text_to_transformer_input(tokenizer, texts: List[str]) -> TensorDataset:
    encoding = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    input_ids = encoding.get('input_ids')
    attention_mask = encoding.get('attention_mask')

    input_values_x = TensorDataset(input_ids, attention_mask)

    return input_values_x


def np_array_to_tensor_dataset(x: np.ndarray) -> TensorDataset:
    if isinstance(x, scipy.sparse.csr_matrix):
        x = x.toarray()
    x = torch.from_numpy(x)
    x = TensorDataset(x)
    
    return x

In [16]:
x_test = convert_text_to_transformer_input(tokenizer, test_fin.Terms.tolist())
x_dev = convert_text_to_transformer_input(tokenizer, dev_fin.Terms.tolist())

In [17]:
unique_list = test_fin['Class'].unique()

dic_count = dict()
count = 0

for el in unique_list:
    count += 1
    dic_count.update({el:count})
    

In [18]:
test_fin['Class_int'] = test_fin['Class'].apply(lambda x: dic_count.get(x))
dev_fin['Class_int'] = dev_fin['Class'].apply(lambda x: dic_count.get(x))

In [21]:
y_test = np_array_to_tensor_dataset(test_fin['Class_int'].values)
y_dev = np_array_to_tensor_dataset(dev_fin['Class_int'].values)

In [23]:
joblib.dump(x_test, './data/x_test.lib')
joblib.dump(y_test, './data/y_test.lib')
joblib.dump(x_dev, './data/x_dev.lib')
joblib.dump(y_dev, './data/y_dev.lib')

['/Storage/Data/Knodle/y_dev.lib']

#  Final Preparations for the train data - the X matrix

analogous to the test data above, except the label translation

In [78]:
train_fin = pd.read_csv('./data/notes_1grams.csv')
train_fin.dropna(inplace=True)

In [79]:
x_train = convert_text_to_transformer_input(tokenizer, train_fin.ngrams.tolist())

In [48]:
joblib.dump(x_train, './data/x_train.lib')

['/Storage/Data/Knodle/x_train.lib']

Now that we have created all our necessary matrices, we will insert all of it into knodle to build our models. The modelling part will subsome in the following file called "task1_modeling_with_knodle"